In [ ]:
### FOR 3 CLASS
# java  -mx2g -cp stanford-ner.jar edu.stanford.nlp.ie.NERServer -port 9199 -loadClassifier english.all.3class.distsim.crf.ser.gz

### FOR 7 CLASS
# java  -mx2g -cp stanford-ner.jar edu.stanford.nlp.ie.NERServer -port 9199 -loadClassifier english.muc.7class.distsim.crf.ser.gz

from sner import Ner

tagger = Ner(host='localhost',port=9199)

In [ ]:
import pandas as pd 
  
df = pd.read_csv("tweets.csv")

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
import re

tok = WordPunctTokenizer()
# pat1 = r'@[A-Za-z0-9]+'
# pat2 = r'https?://[A-Za-z0-9./]+'
pat = r'https?://[A-Za-z0-9./]+'
# combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
#     lower_case = letters_only.lower()
#     words = tok.tokenize(lower_case)
    words = tok.tokenize(letters_only)
    
    return (" ".join(words)).strip()

In [ ]:
df['clean_text'] = df['full_text'].apply(tweet_cleaner)
df.head()

In [ ]:
def get_who(text):

    classified_text = tagger.get_entities(text)
    who = []
    
    for i, j  in classified_text:
        if j == 'PERSON':
            who.append(i)
    
    if not who:
        return ""
    
    whoString = ", ".join(who)
    
    return whoString


def get_where(text):
    
    classified_text = tagger.get_entities(text) 
    where = []
    
    for i, j  in classified_text:
        if j == 'LOCATION':
            where.append(i)

    if not where:
        return ""
    
    whoString = ", ".join(where)
    
    return whoString

In [ ]:
df['who'] = df['clean_text'].apply(get_who)
df['where'] = df['clean_text'].apply(get_where)

In [ ]:
df.head()